In [97]:
import ctypes
from pathlib import Path
import pandas as pd

In [98]:

class MapsForgeHeader(ctypes.BigEndianStructure):
    _pack_ = 1
    _fields_ = [
        ("magic_byte", ctypes.c_char * 20),
        ("header_size", ctypes.c_uint32),
        ("file_version", ctypes.c_uint32),
        ("file_size", ctypes.c_uint64),
        ("date_of_creation", ctypes.c_uint64),
        ("minLat", ctypes.c_int32),
        ("minLon", ctypes.c_int32),
        ("maxLat", ctypes.c_int32),  # most nothern point
        ("maxLon", ctypes.c_int32),
        ("tile_size", ctypes.c_uint16),
    ]


class ParsedFileName:
    country: str
    state: str
    creationDate: str
    minLong: int
    minLat: int
    sizeNorthSouth: int
    sizeWestEast: int

    def __init__(self, filename):
        self.country = filename[:2]
        self.state = filename[2:6]
        self.creationDate = filename[6:12]
        self.field1 = int(filename[12:15], 36)
        self.field2 = int(filename[15:18], 36)
        self.z1 = int(filename[18:21], 36)
        self.z2 = int(filename[21:24], 36)


def parseHeader(file: Path) -> MapsForgeHeader:
    header = MapsForgeHeader.from_buffer_copy(file.read_bytes())
    return header


def parseFilename(filename) -> ParsedFileName:
    return ParsedFileName(filename)


In [99]:
files = [f for f in Path("./igps_maps").glob(("*.map")) if f.is_file()]
data = []


print("--- files ----")
for i in files:
    print(i.name)


--- files ----
Carinthia-AT02002303103DR27W01K00Q.map
Burgenland-AT01002303103FV26Y00R019.map
Vorarlberg-AT08002303103BS27H00H00P.map
UpperAustria-AT04002303103DT26C01G01A.map
Tyrol-AT07002303103C527B01U012.map
Steiermark-AT06002303103EC27801O016.map
LowerAustria-AT03002303103EW26301O01K.map
Vienna-AT09002303103G026S009007.map
Salzburg-AT05002303103DE271018012.map
Slovenia-SI00002303103E628502301C.map
Queensland-AU04002303105KZ3BL0BA0DI.map
Finmark-NO04002303103IP1C707605E.map
GreaterLondon-UK030023031035G23H00J00F.map
Tanzania-TZ00002303103O936E07906V.map


In [100]:
for f in files:
    header = parseHeader(f)
    # fix modified name (my maps are named like Vorarlberg-AT08002303103BS27H00H00P.map)
    humanName, filename = f.name.split("-")
    pf = parseFilename(filename)
    data.append({"file": f, "header": header, "parsedFilename": pf, "Name": humanName})

In [101]:
print("--- bounding boxes from mapsforge files ---")

for entry in data:
    print(
        entry["Name"].ljust(15),
        "minLat",
        str(entry["header"].minLat).ljust(11),
        "minLon",
        str(entry["header"].minLon).ljust(9),
        "maxLat",
        str(entry["header"].maxLat).ljust(11),
        "maxLon",
        entry["header"].maxLon,
    )


--- bounding boxes from mapsforge files ---
Carinthia       minLat 46350000    minLon 12635000  maxLat 47150000    maxLon 15085000
Burgenland      minLat 46810000    minLon 15975000  maxLat 48140000    maxLon 17180000
Vorarlberg      minLat 46837000    minLon 9527000   maxLat 47600000    maxLon 10241000
UpperAustria    minLat 47440000    minLon 12730000  maxLat 48795000    maxLon 15010000
Tyrol           minLat 46630000    minLon 10080000  maxLat 47765000    maxLon 12985000
Steiermark      minLat 46590000    minLon 13545000  maxLat 47850000    maxLon 16190000
LowerAustria    minLat 47400000    minLon 14435000  maxLat 49040000    maxLon 17090000
Vienna          minLat 48114000    minLon 16178000  maxLat 48327000    maxLon 16582000
Salzburg        minLat 46925000    minLon 12055000  maxLat 48060000    maxLon 14015000
Slovenia        minLat 45420810    minLon 13305090  maxLat 46879560    maxLon 16600340
Queensland      minLat -29220000   minLon 137960000 maxLat -9150000    maxLon 15582000

In [102]:
print("--- sorted by maxLat, using field 2 from filename ---")
data = sorted(data, key=lambda entry: entry["header"].maxLat, reverse=True)
prevmaxLatHeader = 0
prevmaxLatFilename = 0
for entry in data:
    maxLatHeader = entry["header"].maxLat
    maxLatFilename = entry["parsedFilename"].field2

    diffHeader =  maxLatHeader - prevmaxLatHeader
    diffFileName = maxLatFilename - prevmaxLatFilename

    print(
        entry["Name"].ljust(13),
        "from header:",
        str(maxLatHeader).ljust(9),
        "diff",
        str(diffHeader).ljust(9) if maxLatHeader != diffHeader else " --- ".ljust(9), 
        "filename:",
        str(maxLatFilename).ljust(9),
        "diff",
        str(diffFileName).ljust(9) if maxLatFilename != diffFileName else " --- ".ljust(9), 
        "diff factor",
        str(diffHeader/diffFileName) if maxLatHeader != diffHeader else " --- ".ljust(9)
    )
    prevmaxLatHeader = maxLatHeader
    prevmaxLatFilename = maxLatFilename



--- sorted by maxLat, using field 2 from filename ---
Finmark       from header: 71420000  diff  ---      filename: 1735      diff  ---      diff factor  ---     
GreaterLondon from header: 51696000  diff -19724000 filename: 2717      diff 982       diff factor -20085.539714867617
LowerAustria  from header: 49040000  diff -2656000  filename: 2811      diff 94        diff factor -28255.31914893617
UpperAustria  from header: 48795000  diff -245000   filename: 2820      diff 9         diff factor -27222.222222222223
Vienna        from header: 48327000  diff -468000   filename: 2836      diff 16        diff factor -29250.0
Burgenland    from header: 48140000  diff -187000   filename: 2842      diff 6         diff factor -31166.666666666668
Salzburg      from header: 48060000  diff -80000    filename: 2845      diff 3         diff factor -26666.666666666668
Steiermark    from header: 47850000  diff -210000   filename: 2852      diff 7         diff factor -30000.0
Tyrol         from header: 

In [103]:
print("--- sorted by minLon---")
data = sorted(data, key=lambda entry: entry["header"].minLon, reverse=True)
prevmaxLonHeader = 0
prevmaxLonFilename = 0
for entry in data:
    maxLonHeader = entry["header"].minLon
    maxLonFilename = entry["parsedFilename"].field1

    diffHeader =  maxLonHeader - prevmaxLonHeader
    diffFileName = maxLonFilename - prevmaxLonFilename

    print(
        entry["Name"].ljust(13),
        "from header:",
        str(maxLonHeader).ljust(9),
        "diff",
        str(diffHeader).ljust(9) if maxLonHeader != diffHeader else " --- ".ljust(9), 
        "filename:",
        str(maxLonFilename).ljust(9),
        "diff",
        str(diffFileName).ljust(9) if maxLonFilename != diffFileName else " --- ".ljust(9), 
        "diff factor",
        str(diffHeader/diffFileName) if maxLonHeader != diffHeader else " --- ".ljust(9)
    )
    prevmaxLonHeader = maxLonHeader
    prevmaxLonFilename = maxLonFilename


--- sorted by minLon---
Queensland    from header: 137960000 diff  ---      filename: 7235      diff  ---      diff factor  ---     
Tanzania      from header: 29243950  diff -108716050 filename: 4761      diff -2474     diff factor 43943.431689571546
Finmark       from header: 20440000  diff -8803950  filename: 4561      diff -200      diff factor 44019.75
Vienna        from header: 16178000  diff -4262000  filename: 4464      diff -97       diff factor 43938.14432989691
Burgenland    from header: 15975000  diff -203000   filename: 4459      diff -5        diff factor 40600.0
LowerAustria  from header: 14435000  diff -1540000  filename: 4424      diff -35       diff factor 44000.0
Steiermark    from header: 13545000  diff -890000   filename: 4404      diff -20       diff factor 44500.0
Slovenia      from header: 13305090  diff -239910   filename: 4398      diff -6        diff factor 39985.0
UpperAustria  from header: 12730000  diff -575090   filename: 4385      diff -13       diff fac

Seems like Longitute is 0-8192 where greenwhich is 4096 (or bit 13 is the signed bit). Every 44*10^-3 degrees the value increases by 1 in the filename